## Decision Tree from Scratch

In [20]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Dataset

- `X_train` contains three features for each example 
    - Brown Color (A value of `1` indicates "Brown" cap color and `0` indicates "Red" cap color)
    - Tapering Shape (A value of `1` indicates "Tapering Stalk Shape" and `0` indicates "Enlarging" stalk shape)
    - Solitary  (A value of `1` indicates "Yes" and `0` indicates "No")

- `y_train` is whether the mushroom is edible 
    - `y = 1` indicates edible
    - `y = 0` indicates poisonous

In [21]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

#### View the variables
Let's get more familiar with your dataset.  
- A good place to start is to just print out each variable and see what it contains.

The code below prints the first few elements of `X_train` and the type of the variable.

In [22]:
print("First few elements of X_train:\n", X_train[:])
print("Type of X_train:",type(X_train))
print(y_train)

First few elements of X_train:
 [[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]
 [0 1 1]
 [0 0 0]
 [1 0 1]
 [0 1 0]
 [1 0 0]]
Type of X_train: <class 'numpy.ndarray'>
[1 1 0 0 1 0 0 1 1 0]


#### Check the dimensions of your variables

Another useful way to get familiar with your data is to view its dimensions.

Please print the shape of `X_train` and `y_train` and see how many training examples you have in your dataset.

In [23]:
print ('The shape of X_train is:', X_train.shape)
print ('The shape of y_train is: ', y_train.shape)
print ('Number of training examples (m):', len(X_train))

The shape of X_train is: (10, 3)
The shape of y_train is:  (10,)
Number of training examples (m): 10


## Split dataset
- Split the dataset to left and right nodes base on the feature


In [24]:
def split_dataset(X: np.ndarray, node_indices, feature): 
    left_indices = []
    right_indices = []
    for i in node_indices: 
        left_indices.append(i) if X[i, feature] == 1 else right_indices.append(i)
    return left_indices, right_indices    

## Compute entropy (measuring purity)

In [25]:
def compute_entropy(y: np.ndarray): 
    entropy = 0.
    n = len(y)
    number_element_1 = np.count_nonzero(y)

    number_element_0 =  n - number_element_1
    if n == 0 or number_element_1 == n or number_element_1 == 0 or number_element_0 == 0: return 0
    
    p1 = number_element_1/n
    po = 1  - p1
    entropy = -p1 * np.log2(p1) - po * np.log2(po)
        
            
        
            
    ### END CODE HERE ###        
    
    return entropy

## Information Gain

In [26]:
def information_gain(X: np.ndarray, y, node_indices, feature ): 
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    
    information_gain = 0
    n_left = len(y_left)
    n_right = len(y_right)
    total_n = n_left + n_right
    
    node_entropy = compute_entropy(y_node)
    left_entropy = compute_entropy(y_left)
    right_entropy = compute_entropy(y_right)    
    w_left = n_left/total_n
    w_right = n_right/total_n
    
    information_gain = node_entropy - (w_left * left_entropy + w_right * right_entropy)
    
    return information_gain

## Choose the feature that split the node best

In [27]:
def get_best_split(X, y, node_indices): 
    num_features = X.shape[1]
    best_feature = -1
    max_information_gain = 0
    
    for feature in range(num_features):
        information_gain_i = information_gain(X, y, node_indices, feature)
        if max_information_gain < information_gain_i: 
            best_feature = feature
            max_information_gain =  information_gain_i
    return best_feature
            

## Building the tree

In [28]:
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth): 
    
    # Maximum depth reached - stop splitting
    if current_depth == max_depth: 
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
    
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices)
    formatting = '-'*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))
    
    # Continue slitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [29]:
print(X_train, y_train)

[[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]
 [0 1 1]
 [0 0 0]
 [1 0 1]
 [0 1 0]
 [1 0 0]] [1 1 0 0 1 0 0 1 1 0]


In [30]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)


 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
